In [1]:
import pandas as pd
import numpy as np
import torch
import random
from sklearn.decomposition import PCA, NMF
from torchnmf.nmf import NMFD
from torchnmf.metrics import kl_div
import matplotlib.pyplot as plt

In [2]:
def NFM_decks(df):
    decks = df.filter(regex='^deck_', axis=1)
    decks = decks.drop(["deck_Island", "deck_Swamp", "deck_Mountain", "deck_Plains", "deck_Forest"], axis=1)
    data = decks.to_numpy(dtype=np.float64)
    model = NMF(n_components=40, init='random', max_iter=1000)
    w = model.fit_transform(data)
    return model.components_

In [3]:
def score(list):
    avg = list.mean()
    stddev = list.std()
    return (list - avg)/stddev

In [4]:
def getWinrates(g, sample):
    decks = sample.filter(regex='^deck_', axis=1)
    decks = decks.drop(["deck_Island", "deck_Swamp", "deck_Mountain", "deck_Plains", "deck_Forest"], axis=1)
    won = sample[["won"]]
    d = decks.to_numpy()
    deck_proj = np.matmul(d, g.T)
    totals = np.linalg.norm(deck_proj,ord=1, axis=1)
    for i in range(deck_proj.shape[0]):
        deck_proj[i] /= totals[i]
    winrates = np.matmul(won.T, deck_proj)/np.matmul(np.ones(won.shape[0]), deck_proj)
    return winrates.to_numpy()[0]

In [5]:
df_iter = pd.read_csv("game_data_public.VOW.PremierDraft.csv", chunksize=1000)
sample = next(df_iter)
decks = sample.filter(regex='^deck_', axis=1)
won = sample[["won"]]
decks = decks.drop(["deck_Island", "deck_Swamp", "deck_Mountain", "deck_Plains", "deck_Forest"], axis=1)
comms = [NFM_decks(next(df_iter)) for i in range(3)]

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
def prettyPrint(comp):
    threshold = 0.03
    min_count = 5
    max_count = 15
    items = list(zip(comp, decks.columns))
    items.sort(reverse=True)
    print("Group:", items[0][1][5:])
    count = 0
    for value, name in items:
        if (value > threshold or count < min_count) and count < max_count:
            print("\t", name[5:], value)
            count += 1
prettyPrint(comms[0][0])

Group: Flame-Blessed Bolt
	 Flame-Blessed Bolt 1.8530482875769068
	 Belligerent Guest 0.28792851962600896
	 Lightning Wolf 0.11898060818889389
	 Stormchaser Drake 0.056196917257576647
	 Kessig Flamebreather 0.051319585244925976
	 Reckless Impulse 0.0358681939225941
	 Wandering Mind 0.030862794489841708


In [7]:
def deck_iter(size):
    while True:
        df_iter = pd.read_csv("game_data_public.VOW.PremierDraft.csv", chunksize=size, low_memory=False)
        for item in df_iter:
            if(item.shape[0] == size):
                yield item

In [13]:
d_iter = iter(deck_iter(20000))
            
def geneticStep(d1, d2, keep=0.95, win_weight = 0.5):
    matches = []
    ret = 0.0 * d1
    thresh = min(0.03, (1-keep)**2)
    d2_indecies = set(range(d2.shape[0]))
    for i in range(d1.shape[0]):
        min_match = 1000
        min_index = 0
        for j in d2_indecies:
            n1 = d1[i]
            n1 /= np.linalg.norm(n1, ord=1)
            n1 = (n1 > thresh) * n1
            n2 = d2[j]
            n2 /= np.linalg.norm(n2, ord=1)
            n2 = (n2 > thresh) * n2
            if max(n1) > thresh and max(n2) > thresh:
                match = np.linalg.norm(n1 - n2, ord=1)
            else:
                print(n1,n2)
                match = 100
            if match < min_match and match > thresh:
                min_match = match
                min_index = j
        for j in range(i+1, d1.shape[0]):
            n1 = d1[i]
            n1 /= np.linalg.norm(n1, ord=1)
            n1 = (n1 > thresh) * n1
            n2 = d1[j]
            n2 /= np.linalg.norm(n2, ord=1)
            n2 = (n2 > thresh) * n2
            if max(n1) > thresh and max(n2) > thresh:
                match = np.linalg.norm(n1 - n2, ord=1)
            else:
                match = 100
            if match < min_match:
                min_match = match
                d1[j], d2[min_index] = d2[min_index], d1[j]
        d2_indecies.remove(min_index)
        matches.append([min_match, i, min_index])
    match_score = [match[0] for match in matches]
    match_score = score(np.array(match_score))
    win_rates = getWinrates(d1, next(d_iter))
    win_scores = score(win_rates)
    for i in range(len(matches)):
        matches[i][0] = win_weight * win_scores[i] + (1 - win_weight) * match_score[i]
    matches.sort(reverse=True)
    for i, vals in enumerate(matches):
        match, ind1, ind2 = vals
        n1 = d1[ind1]
        n1 = (n1 > thresh) * n1
        n1 /= np.linalg.norm(n1, ord=1)
        n2 = d2[ind2]
        n2 = (n2 > thresh) * n2
        n2 /= np.linalg.norm(n2, ord=1)
        
        ratio = (.85*(i)/(len(matches)))**10
        weight = (1-keep)*(1-ratio) + keep * ratio
        if weight > 1:
            raise ValueError("invalid weight %d" % (weight))
        if weight < 0.5:
            ret[i] = (n1 * (1 - weight)) + (n2 * weight)
        else:
            ret[i] = random.choice([n1,n2])
    return ret, win_rates
g1, w1  = geneticStep(comms[0], comms[1])
prettyPrint(g1[0])
g2, w2 = geneticStep(g1, comms[2])
prettyPrint(g2[0])
g3, w3  = geneticStep(g2, comms[0])
print(w3[0])
prettyPrint(g2[0])

Group: Sigarda's Imprisonment
	 Sigarda's Imprisonment 0.4032733850508619
	 Faithbound Judge 0.05755696801666963
	 Brine Comber 0.048351173908376364
	 Heron-Blessed Geist 0.031216974088510235
	 Cruel Witness 0.02968706470683667
Group: Flame-Blessed Bolt
	 Flame-Blessed Bolt 0.6933826103623814
	 Belligerent Guest 0.1038471877700498
	 Lightning Wolf 0.042912739507832594
	 Stormchaser Drake 0.020864536761924412
	 Kessig Flamebreather 0.01850943634251052
0.579237788193252
Group: Flame-Blessed Bolt
	 Flame-Blessed Bolt 0.6933826103623814
	 Belligerent Guest 0.1038471877700498
	 Lightning Wolf 0.042912739507832594
	 Stormchaser Drake 0.020864536761924412
	 Kessig Flamebreather 0.01850943634251052


In [14]:
df_iter = iter(deck_iter(1000))
print("Begin Generation", 0)
g_n = NFM_decks(next(df_iter))
max_gen = 2000
for generation, match in enumerate(df_iter):
    deck_NFM = NFM_decks(match)
    if generation < 100:
        gen_keep = 1/(generation + 2)
        print(gen_keep)
    else:
        gen_keep = 1/100
    g_n, w_n = geneticStep(g_n, deck_NFM, keep=(1 - gen_keep), win_weight=0.5*generation/max_gen)
    if generation == max_gen:
        break
    if generation % 15 == 0:
        print("Begin Generation", generation + 1)
        for i in range(5):
            print(w_n[i])
            prettyPrint(g_n[i])
for i in range(5):
    prettyPrint(g_n[i])

Begin Generation 0
0.5
Begin Generation 1
0.5538235422056491
Group: Reckless Impulse
	 Reckless Impulse 0.665477513744445
	 Alluring Suitor 0.08658577993264994
	 Lacerate Flesh 0.0824102512635044
	 Rending Flame 0.058159119764822534
	 Blood Fountain 0.0550514937327026
	 Belligerent Guest 0.052315841561875534
0.5477024728370699
Group: Ragged Recluse
	 Ragged Recluse 0.5796356315830892
	 Blood Fountain 0.09355412016241536
	 Fell Stinger 0.0677808020656133
	 Skulking Killer 0.059197544138730324
	 Parasitic Grasp 0.05837228737742304
	 Undying Malice 0.0515836477610477
	 Pointed Discussion 0.048587629960638395
	 Rending Flame 0.04128833695104269
0.5527955339895746
Group: Doomed Dissenter
	 Doomed Dissenter 0.26147120426803544
	 Fell Stinger 0.11226418382834844
	 Rot-Tide Gargantua 0.10598709006025972
	 Grisly Ritual 0.10529281213151684
	 Gift of Fangs 0.09643441618374547
	 Desperate Farmer 0.08593034927584126
	 Bloodsworn Squire 0.0788617589932323
	 Hero's Downfall 0.05894174762324968
	 Wan

0.015873015873015872
0.015625
0.015384615384615385
0.015151515151515152
0.014925373134328358
0.014705882352941176
0.014492753623188406
0.014285714285714285
0.014084507042253521
0.013888888888888888
0.0136986301369863
0.013513513513513514
0.013333333333333334
0.013157894736842105
0.012987012987012988
Begin Generation 76
0.5516609235479383
Group: Diregraf Scavenger
	 Diregraf Scavenger 0.624397520867033
	 Undead Butler 0.051587117192844305
	 Sorin the Mirthless 0.030347339697266662
	 Rending Flame 0.02933411702265282
	 Headless Rider 0.027466412803830356
0.5488503807429612
Group: Alluring Suitor
	 Alluring Suitor 0.13946367050630945
	 Sure Strike 0.08901677783042752
	 Circle of Confinement 0.08248307583595843
	 Evolving Wilds 0.0751987492428662
	 Estwald Shieldbasher 0.07460188712535509
	 Rending Flame 0.06686850250624635
	 Blood Hypnotist 0.053434582680545094
	 Militia Rallier 0.0509099734761562
	 Markov Waltzer 0.04838156316430606
	 Vampire Slayer 0.04623010460532916
	 Arm the Cathars 

Begin Generation 151
0.5457360284453827
Group: Fearful Villager
	 Fearful Villager 0.44549940151234263
	 Rending Flame 0.06281846406515718
	 Voltaic Visionary 0.04557005756973797
	 Sure Strike 0.03236334974878985
	 Weary Prisoner 0.031923379187497866
0.5621424193917366
Group: Fierce Retribution
	 Fierce Retribution 0.39065497954103784
	 Sigarda's Imprisonment 0.03803941986682471
	 Twinblade Geist 0.03789985443527238
	 Kindly Ancestor 0.03614536740895758
	 Fear of Death 0.03300032446791005
0.5422227232641182
Group: Voldaren Epicure
	 Voldaren Epicure 0.4114901694826838
	 Bloody Betrayal 0.06624487964479003
	 Reckless Impulse 0.06324766842124259
	 Vampires' Vengeance 0.06319216645244712
	 Alluring Suitor 0.04863438040435776
	 Desperate Farmer 0.03686447580570864
	 Kessig Flamebreather 0.036318073502237494
0.5493556108844195
Group: Blood Petal Celebrant
	 Blood Petal Celebrant 0.49806475989818616
	 Ballista Watcher 0.0436912440380151
	 Hero's Downfall 0.038472198654806174
	 Undying Malice

Begin Generation 226
0.545962518718457
Group: Bleed Dry
	 Bleed Dry 0.49785766698549877
	 Diregraf Scavenger 0.08229422243492694
	 Headless Rider 0.03215807356993509
	 Falkenrath Celebrants 0.027663122236376656
	 Volatile Arsonist 0.02693462218454319
0.5390632084480281
Group: Ancestral Anger
	 Ancestral Anger 0.3358851483580422
	 Alluring Suitor 0.06054481018237042
	 Bloodtithe Harvester 0.046854638305263256
	 Ceremonial Knife 0.03689444293229365
	 Daybreak Combatants 0.03292662633743068
0.5567625612397085
Group: Courier Bat
	 Courier Bat 0.38737821637498404
	 Mindleech Ghoul 0.05506332786008533
	 Undying Malice 0.04532004091638278
	 Vampire's Kiss 0.044555043485903716
	 Rot-Tide Gargantua 0.03687636691384547
	 Rural Recruit 0.030704954145711366
0.5408308622684422
Group: Kindly Ancestor
	 Kindly Ancestor 0.5273842162694463
	 Parish-Blade Trainee 0.06881427313040457
	 Drogskol Infantry 0.032979249531444224
	 Adamant Will 0.030007652353398293
	 Circle of Confinement 0.026359978496167853


Begin Generation 316
0.5683374901766186
Group: Bleed Dry
	 Bleed Dry 0.3521372994727265
	 Flourishing Hunter 0.09586667041378338
	 Diregraf Scavenger 0.05704402116047349
	 Aim for the Head 0.04751175556871747
	 Gluttonous Guest 0.04481423365744291
	 Restless Bloodseeker 0.031752396414722536
0.5754743560839077
Group: Voldaren Epicure
	 Voldaren Epicure 0.23816354137684212
	 Parasitic Grasp 0.06946280190092664
	 Lacerate Flesh 0.05864373876801376
	 Ballista Watcher 0.04502918431469312
	 Reckless Impulse 0.04179984392675937
	 Bloodtithe Harvester 0.03239152997301721
0.5486545662450687
Group: Bloodcrazed Socialite
	 Bloodcrazed Socialite 0.2516144970938861
	 Evolving Wilds 0.1379305219372861
	 Bloodtithe Harvester 0.08760908298821804
	 Ragged Recluse 0.06770907597747287
	 Desperate Farmer 0.059515708615486315
	 Courier Bat 0.048548097996282354
	 Vampire's Kiss 0.04204143639382432
	 Bloody Betrayal 0.035083600182624367
0.5536561752219576
Group: Chill of the Grave
	 Chill of the Grave 0.2603

...

Begin Generation 1621
0.5524645660366342
Group: Flame-Blessed Bolt
	 Flame-Blessed Bolt 0.4465218973792114
	 Blood Petal Celebrant 0.05808406355453525
	 Alluring Suitor 0.05404502601870465
	 Voldaren Epicure 0.036681567853604234
	 Lacerate Flesh 0.029227379410558398
0.5386319194877984
Group: Traveling Minister
	 Traveling Minister 0.4831892265453986
	 Vampire Slayer 0.08229404090602613
	 Daybreak Combatants 0.056918587778835746
	 Markov Waltzer 0.03644165950859528
	 Fleeting Spirit 0.03378205803934748
0.5468824486621731
Group: Falkenrath Celebrants
	 Falkenrath Celebrants 0.5862406846765789
	 Undying Malice 0.06324474148663277
	 Evolving Wilds 0.03683422317492792
	 Doomed Dissenter 0.02523440633112132
	 Ragged Recluse 0.019128598689927702
0.5355476751827609
Group: Ragged Recluse
	 Ragged Recluse 0.31709779383559666
	 Voldaren Epicure 0.06605413607809626
	 Courier Bat 0.059857991642827624
	 Edgar's Awakening 0.050493212090145984
	 Hero's Downfall 0.03988955620386612
	 Bloodtithe Harvest

Begin Generation 1711
0.5638466637834382
Group: Hungry Ridgewolf
	 Hungry Ridgewolf 0.15257265100616166
	 Ill-Tempered Loner 0.08426276419931973
	 Flame-Blessed Bolt 0.07579128337554845
	 Runebound Wolf 0.053853298013026914
	 Ballista Watcher 0.05150277988241417
	 Abrade 0.05042853555736895
	 Magma Pummeler 0.04489795227985785
	 Parasitic Grasp 0.04440350481882841
	 Frenzied Devils 0.04425908536204911
	 Alluring Suitor 0.043760295797092556
	 Grisly Ritual 0.0431717142784329
	 Edgar's Awakening 0.042578684352010586
	 Restless Bloodseeker 0.042487527263833376
	 Aim for the Head 0.0402639477895093
	 Falkenrath Forebear 0.037654426794726345
0.5401925204839416
Group: Abrade
	 Abrade 0.5197510304099883
	 Voltaic Visionary 0.06920670002284703
	 Falkenrath Celebrants 0.06353314709464954
	 Flame-Blessed Bolt 0.04909692199165201
	 Restless Bloodseeker 0.04483264503564803
	 Ill-Tempered Loner 0.03253922138592805
0.5487167435304102
Group: Drogskol Infantry
	 Drogskol Infantry 0.3043428154879493
	 

Begin Generation 1801
0.5699652238131785
Group: Fearful Villager
	 Fearful Villager 0.22177284934864094
	 Sure Strike 0.10836200785943874
	 Child of the Pack 0.059783997710650016
	 Runebound Wolf 0.052572502611373986
	 Reckless Impulse 0.038400852075429565
	 Ballista Watcher 0.03506501349185977
	 Ill-Tempered Loner 0.03469267472257891
	 Wolfkin Outcast 0.034015548263985576
	 Sporeback Wolf 0.03183043865213184
0.5561736869916066
Group: Bloodtithe Harvester
	 Bloodtithe Harvester 0.29663653834120807
	 Desperate Farmer 0.1048774820264526
	 Voldaren Epicure 0.10237834027028067
	 Undying Malice 0.07116675490961029
	 Rot-Tide Gargantua 0.05426649646849097
	 Reckless Impulse 0.03889108480946221
	 Wedding Invitation 0.03577888106432726
0.5610652725647126
Group: Falkenrath Celebrants
	 Falkenrath Celebrants 0.413140578801234
	 Reckless Impulse 0.11421624208097601
	 Lacerate Flesh 0.050597637471488155
	 Alluring Suitor 0.04582114350957633
	 Sure Strike 0.04405001657245437
	 Kessig Wolfrider 0.03

Begin Generation 1876
0.5598668057265586
Group: Abrade
	 Abrade 0.5576391793809296
	 Voltaic Visionary 0.06220508237415611
	 Runebound Wolf 0.04499060984623372
	 Magma Pummeler 0.04070028725331491
	 Blood Servitor 0.033194227173655735
	 Falkenrath Celebrants 0.03121014787233712
0.5684654662636874
Group: Fierce Retribution
	 Fierce Retribution 0.31908495811967613
	 Arm the Cathars 0.06310654364616623
	 Piercing Light 0.059326444679165334
	 Twinblade Geist 0.058974661473262475
	 Valorous Stance 0.039647898637401105
	 Gryffwing Cavalry 0.03532909260758127
	 Panicked Bystander 0.03173974602402859
0.5609856137054151
Group: Kindly Ancestor
	 Kindly Ancestor 0.44657285301813165
	 Heron of Hope 0.11324343349374098
	 Fierce Retribution 0.04749099283725655
	 Adamant Will 0.035396219230686615
	 Unholy Officiant 0.029596116490447123
0.5538597393639352
Group: Ancestral Anger
	 Ancestral Anger 0.23370769427002017
	 Runebound Wolf 0.08980271880347668
	 Abrade 0.0878520625799445
	 Lambholt Raconteur 0

Begin Generation 1966
0.5366932791473563
Group: Fierce Retribution
	 Fierce Retribution 0.41459069745795507
	 Nurturing Presence 0.07525297296905578
	 Sigarda's Imprisonment 0.0722782617249202
	 Nebelgast Beguiler 0.0655772536264456
	 Distracting Geist 0.03929872717925027
	 Circle of Confinement 0.034912717138863764
	 Katilda, Dawnhart Martyr 0.034124500779111576
	 Ceremonial Knife 0.030637319433526752
0.5395949393606817
Group: Flame-Blessed Bolt
	 Flame-Blessed Bolt 0.43909039941066214
	 Falkenrath Celebrants 0.14740697904679342
	 Doomed Dissenter 0.05419365873600099
	 Daybreak Combatants 0.04688009980152605
	 Blood Hypnotist 0.03630123783900331
0.5427873956038581
Group: Sigarda's Imprisonment
	 Sigarda's Imprisonment 0.39868203450178114
	 Kindly Ancestor 0.06281717989188469
	 Markov Purifier 0.05366238223755939
	 Heron-Blessed Geist 0.034659550084926424
	 Angelic Quartermaster 0.032915134302686985
0.5565906910215669
Group: Ancestral Anger
	 Ancestral Anger 0.43159961300253
	 Chill of

In [17]:
for i in range(g_n.shape[0]):
    print(i)
    print(w_n[i])
    prettyPrint(g_n[i])

0
0.5662475233194115
Group: Abrade
	 Abrade 0.3933746472261327
	 Falkenrath Celebrants 0.1334477653454431
	 Voldaren Epicure 0.09326113848137846
	 Vampires' Vengeance 0.03709928334428396
	 Belligerent Guest 0.023679045604702808
1
0.5545407387208307
Group: Ancestral Anger
	 Ancestral Anger 0.3648917848715037
	 Runebound Wolf 0.0840902985048227
	 Belligerent Guest 0.06794438122341005
	 Reckless Impulse 0.05578565385801966
	 Blood Servitor 0.04697657638886644
	 Lambholt Raconteur 0.04295399639464905
	 Flame-Blessed Bolt 0.040953142119997794
	 Cradle of Safety 0.032970877712869134
2
0.5549411195418001
Group: Sanguine Statuette
	 Sanguine Statuette 0.10021573322866284
	 Lambholt Raconteur 0.0894455002348827
	 Lunar Rejection 0.07606528441571309
	 Kessig Flamebreather 0.07074753988810249
	 Wandering Mind 0.06911957367388695
	 Syphon Essence 0.0678436611679119
	 Magma Pummeler 0.05164630766077977
	 Voldaren Epicure 0.04776438912136609
	 Thirst for Discovery 0.04741268533720041
	 Dollhouse of 

In [ ]:
import matplotlib.pyplot as plt
g_0 = -1*g_n[0]
g_0.sort()
plt.plot(-1*g_0)

In [ ]:

d.shape

In [ ]:
tmp = winrates
tmp.sort()
tmp